# Building a GPT, prettified by Andrej Muhic
All credit to
Companion notebook to the [Zero To Hero](https://karpathy.ai/zero-to-hero.html) video on GPT by Andrej Karpathy.

## Key techiques so deep learning is easier to use
- Adam/AdamW optimizer
- [Transformers](https://arxiv.org/abs/1706.03762)
- Residual connections
- Dropout
- Layer/batch normalization
- Automatic differentiation improvements
- Polished libraries that are easy to use, https://pytorch.org/
- Hardware acceleration, cuda, tpus, ...

## Strict run time named dimension checking
- <https://kidger.site/thoughts/jaxtyping/>
- <https://docs.kidger.site/jaxtyping/api/array/>
- <https://github.com/patrick-kidger/jaxtyping>
- <https://github.com/agronholm/typeguard>

## Tracking progress of your runs

- <https://github.com/aimhubio/aim> free
- <https://pytorch.org/tutorials/recipes/recipes/tensorboard_with_pytorch.html> free
- <https://wandb.ai/site> paid

## Optimization, AdamW state of the art
- <https://www.ruder.io/optimizing-gradient-descent/#adam>
- <https://iclr-blogposts.github.io/2023/blog/2023/adamw> as proximal operator
- <https://arxiv.org/abs/2404.04454> Implicit Bias of AdamW: ℓ∞ Norm Constrained Optimization
`optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)`
Adam is just using exponential weighted average of gradient and "variance of gradients", corrected for bias for starting at 0. 
AdamW adds a trick to do better incorporate L2 regularization for decay rate, best views as proximal operator in my opinion.

### Troubles with regular techniques
- Memory constraints even for Adam(W) we need to store two additional vectors thus we 3x our memory requirements.
- Back stepping is not feasible for deep learning, too costly to store harder to parallelize!
- <https://en.wikipedia.org/wiki/Limited-memory_BFGS> approximates inverse of Hessian implicitly and even this is too costly.
- <https://en.wikipedia.org/wiki/Wolfe_conditions>
- <https://en.wikipedia.org/wiki/Backtracking_line_search> more advanced step size

## Visualizing the network
- My personal favourite: <https://github.com/mert-kurttutan/torchview> nicer graph, device='meta' is buggy sometimes, use cuda or cpu.
- Honourable mentions: <https://github.com/szagoruyko/pytorchviz>

## Activations
- <https://www.ai-contentlab.com/2023/03/swishglu-activation-function.html>
- <https://pytorch.org/docs/stable/generated/torch.nn.SiLU.html>

## Useful tricks
### Einsum formulas for humans
- <https://einops.rocks/api/einsum>

### Broadcasting
- <https://numpy.org/doc/stable/user/basics.broadcasting.html> My advice would be if you are not sure do manual broadcasting and test.
- Strictly test dimenstions with something like jaxtyping

### Sanity checks and data
- Inspect your data before you start training
- Overift on small piece of data using simplest possible model before trying to do something fancy
- Getting better and more data will bring larger improvements than using superior model

## NLP specific info

### Tokenization
- In practice one would use something like [Byte Pair Encoding](https://github.com/karpathy/minbpe) or [WordPiece, SentencePiece](https://huggingface.co/docs/transformers/en/tokenizer_summary)
## Attention and geometric deep learning
- Attention: <https://magazine.sebastianraschka.com/p/understanding-and-coding-self-attention>
- [GNN](https://arxiv.org/abs/2301.08210) and [Geometric Deep Learning](https://geometricdeeplearning.com/lectures/)
- Geometric Deep learning excites me as mathematician as it is a very nice way of infusing domain priors, symmetry, geometry of the problem

THe full blown image of the multi head attention with learnable positional encoding. Code follows after. It is based on Karpathy notebook with my scratchpad like notes to facilitate deeper understanding.
![image](multihead_transformer_with_learnable_positional_encoding.svg)

In [1]:
# Let us do some stricter typing
%config Completer.use_jedi = False
# from sklearn.model_selection import train_test_split
import typing
from typing import TYPE_CHECKING, Any, Optional
import torch

# Strict run time dimension checking
# https://kidger.site/thoughts/jaxtyping/
# https://docs.kidger.site/jaxtyping/api/array/
# https://github.com/patrick-kidger/jaxtyping
# https://github.com/agronholm/typeguard

# Tracking the progress: 
#                        https://github.com/aimhubio/aim
#                        https://pytorch.org/tutorials/recipes/recipes/tensorboard_with_pytorch.html
#                        https://wandb.ai/site

# Something on order of distributions in cross entropy and KL
# https://agustinus.kristia.de/techblog/2016/12/21/forward-reverse-kl/

# Visualizing the network
# https://github.com/szagoruyko/pytorchviz
# https://github.com/mert-kurttutan/torchview nicer graph

# Optimization, AdamW state of the art
# https://www.ruder.io/optimizing-gradient-descent/#adam
# https://iclr-blogposts.github.io/2023/blog/2023/adamw/ as proximal operator
# https://arxiv.org/abs/2404.04454 Implicit Bias of AdamW: ℓ∞ Norm Constrained Optimization
# optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)
# exponential weighted average of gradient and "variance of gradients", corrected for bias for starting at 0 
# +  a trick to do better than L2 for decay rate
# Back stepping
# Not feasible for deep learning, too costly to store harder to parallelize!
# https://en.wikipedia.org/wiki/Limited-memory_BFGS approximates inverse of Hessian implicitly and even this is too costly
# https://en.wikipedia.org/wiki/Wolfe_conditions
# https://en.wikipedia.org/wiki/Backtracking_line_search more advanced step size

# Activations
# https://www.ai-contentlab.com/2023/03/swishglu-activation-function.html
# https://pytorch.org/docs/stable/generated/torch.nn.SiLU.html

# Einsum formulas for human, thanks for the hint guys!
# https://einops.rocks/api/einsum/

# Broadcasting
# https://numpy.org/doc/stable/user/basics.broadcasting.html My advice would be if you are not sure do manual broadcasting and test.
# Strictly test dimenstions with something like jaxtyping

# key enabling techiques that made deep learning more easy to work with
# - adam/adamw optimizer
# - residual connections
# - dropout
# - layer/batch normalization
# - automatic differentiation

from jaxtyping import Float, Int64

Array: typing.TypeAlias = torch.Tensor
Long: typing.TypeAlias = Int64

In [2]:
# We always start with a dataset to train on. Let's download the tiny shakespeare dataset
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2024-04-24 11:09:39--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.108.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt.23’

input.txt.23        100%[===================>]   1.06M  --.-KB/s    in 0.07s   

2024-04-24 11:09:39 (15.7 MB/s) - ‘input.txt.23’ saved [1115394/1115394]



In [3]:
# read it in to inspect it
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [4]:
print("length of dataset in characters: ", len(text))

length of dataset in characters:  1115394


In [5]:
# let's look at the first 1000 characters
print(text[:1000])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for revenge.



In [6]:
# here are all the unique characters that occur in this text
chars: list[str] = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


In [7]:
# create a mapping from characters to integers
stoi: dict[str, int] = {ch: i for i, ch in enumerate(chars)}
itos: dict[int, str] = {i: ch for i, ch in enumerate(chars)}
encode = lambda s: [stoi[c] for c in s]  # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l])  # decoder: take a list of integers, output a string

# In practice one would use something like [Byte Pair Encoding](https://github.com/karpathy/minbpe) or [WordPiece, SentencePiece](https://huggingface.co/docs/transformers/en/tokenizer_summary)

print(encode("hii there"))
print(decode(encode("hii there")))

[46, 47, 47, 1, 58, 46, 43, 56, 43]
hii there


In [8]:
# let's now encode the entire text dataset and store it into a torch.Tensor
import torch  # we use PyTorch: https://pytorch.org

data = torch.tensor(encode(text), dtype=torch.int64)
print(data.shape, data.dtype)
print(data[:1000])  # the 1000 characters we looked at earier will to the GPT look like this

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59,  1, 39, 56, 43,  1, 39, 50, 50,
         1, 56, 43, 57, 53, 50, 60, 43, 42,  1, 56, 39, 58, 46, 43, 56,  1, 58,
        53,  1, 42, 47, 43,  1, 58, 46, 39, 52,  1, 58, 53,  1, 44, 39, 51, 47,
        57, 46, 12,  0,  0, 13, 50, 50, 10,  0, 30, 43, 57, 53, 50, 60, 43, 42,
         8,  1, 56, 43, 57, 53, 50, 60, 43, 42,  8,  0,  0, 18, 47, 56, 57, 58,
         1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 18, 47, 56, 57, 58,  6,  1, 63,
        53, 59,  1, 49, 52, 53, 61,  1, 15, 39, 47, 59, 57,  1, 25, 39, 56, 41,
      

In [9]:
# Let's now split up the data into train and validation sets
n = int(0.9 * len(data))  # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

In [10]:
block_size = 8
# First block and the target
train_data[:block_size + 1]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [11]:
# This is what we want to learn for max_context_size = block_size
# It is important that we train also for shorter sequences than block_size
# When creating batches in practice we could need padding token and if pad left or right!
# Also how to shuffle if we need to if data does not fit in memory: https://blog.janestreet.com/how-to-shuffle-a-big-dataset/
x = train_data[:block_size]
y = train_data[1:block_size + 1]
for t in range(block_size):
    context = x[:t + 1]
    target = y[t]
    print(f"when input is {context} the target: {target}")

when input is tensor([18]) the target: 47
when input is tensor([18, 47]) the target: 56
when input is tensor([18, 47, 56]) the target: 57
when input is tensor([18, 47, 56, 57]) the target: 58
when input is tensor([18, 47, 56, 57, 58]) the target: 1
when input is tensor([18, 47, 56, 57, 58,  1]) the target: 15
when input is tensor([18, 47, 56, 57, 58,  1, 15]) the target: 47
when input is tensor([18, 47, 56, 57, 58,  1, 15, 47]) the target: 58


In [12]:
torch.manual_seed(1337)
batch_size = 4  # how many independent sequences will we process in parallel?
block_size = 8  # what is the maximum context length for predictions?


def get_batch(split_kind: str) -> tuple[Int64[Array, "n_batches block_size"], Int64[Array, "n_batches block_size"]]:
    # generate a small batch of data of inputs x and targets y
    data = train_data if split_kind == 'train' else val_data
    # Random starting indices of blocks, notice that blocks can overlap
    # To do something like this for real: 
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i + block_size] for i in ix])
    y = torch.stack([data[i + 1:i + block_size + 1] for i in ix])
    return x, y


xb, yb = get_batch('train')
print('inputs:')
print(xb.shape)
print(xb)
print('targets:')
print(yb.shape)
print(yb)

print('----')

for b in range(batch_size):  # batch dimension
    for t in range(block_size):  # time dimension
        context = xb[b, :t + 1]
        target = yb[b, t]
        print(f"when input is {context.tolist()} the target: {target}")

inputs:
torch.Size([4, 8])
tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])
targets:
torch.Size([4, 8])
tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])
----
when input is [24] the target: 43
when input is [24, 43] the target: 58
when input is [24, 43, 58] the target: 5
when input is [24, 43, 58, 5] the target: 57
when input is [24, 43, 58, 5, 57] the target: 1
when input is [24, 43, 58, 5, 57, 1] the target: 46
when input is [24, 43, 58, 5, 57, 1, 46] the target: 43
when input is [24, 43, 58, 5, 57, 1, 46, 43] the target: 39
when input is [44] the target: 53
when input is [44, 53] the target: 56
when input is [44, 53, 56] the target: 1
when input is [44, 53, 56, 1] the target: 58
when input is [44, 53, 56, 1, 58] the target: 46
when input is [44, 53

In [13]:
print(xb)  # our input to the transformer

tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])


In [14]:
import torch
import torch.nn as nn
from torch.nn import functional as F

torch.manual_seed(1337)


class BigramLanguageModel(nn.Module):

    def __init__(self, vocab_size: int):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx: Long[torch.Tensor, "batch_dim context_dim"],
                targets: Optional[Long[torch.Tensor, "batch_dim context_dim"]] = None):

        # idx and targets are both (B,T) tensor of integers
        logits: Long[torch.Tensor, "batch_dim context_dim latent_dim"] = self.token_embedding_table(
            idx)  # (B,T,C=vocab_size)

        if targets is None:
            return logits, None
        else:
            # Note that here strictly speaking this does not fix batch size explicitly to B
            B, T, C = logits.shape  # (B,T,C=vocab_size)
            # Just a hack to avoid transposing, cross_entropy expects B x C x T in batched mode
            # This converts into non batched mode
            logits: Long[torch.Tensor, "batch_dim*context_dim latent_dim"] = logits.view(B * T, C)
            targets: Long[torch.Tensor, "batch_dim*context_dim"] = targets.view(B * T)
            # https://agustinus.kristia.de/techblog/2016/12/21/forward-reverse-kl/
            loss: Float[torch.Tensor, ""] = F.cross_entropy(logits, targets)
            return logits, loss

    def generate(self, idx: Long[torch.Tensor, "batch_dim context_dim"], max_new_tokens: int):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self(idx)
            # focus only on the last time step
            logits = logits[:, -1, :]  # becomes (B, vocab_size)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1)  # (B, vocab_size)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1)  # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1)  # (B, T+1)
            # What can go wrong here? and it is not handled at all
        return idx


m = BigramLanguageModel(vocab_size)
logits, loss = m(xb, yb)
print(logits.shape)
print(loss)
print(loss.shape)
print(decode(m.generate(idx=torch.zeros((1, 1), dtype=torch.long), max_new_tokens=100)[0].tolist()))

torch.Size([32, 65])
tensor(4.8786, grad_fn=<NllLossBackward0>)
torch.Size([])

Sr?qP-QWktXoL&jLDJgOLVz'RIoDqHdhsV&vLLxatjscMpwLERSPyao.qfzs$Ys$zF-w,;eEkzxjgCKFChs!iWW.ObzDnxA Ms$3


In [15]:
# create a PyTorch optimizer
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)
# weighted average of mean and "variance" of gradients +  a trick
# Not feasible for deep learning, too costly to store harder to parallelize!
# https://en.wikipedia.org/wiki/Limited-memory_BFGS approximates inverse of Hessian implicitly and even this is too costly
# https://en.wikipedia.org/wiki/Wolfe_conditions
# https://en.wikipedia.org/wiki/Backtracking_line_search more advanced step size

In [16]:
batch_size = 32
n_steps = 1_000
for steps in range(n_steps):  # increase number of steps for good results...

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print(loss.item())

3.7218432426452637


In [17]:
print(decode(m.generate(idx=torch.zeros((1, 1), dtype=torch.long), max_new_tokens=500)[0].tolist()))


olylvLLko'TMyatyIoconxad.?-tNSqYPsx&bF.oiR;BD$dZBMZv'K f bRSmIKptRPly:AUC&$zLK,qUEy&Ay;ZxjKVhmrdagC-bTop-QJe.H?x
JGF&pwst-P sti.hlEsu;w:w a BG:tLhMk,epdhlay'sVzLq--ERwXUzDnq-bn czXxxI&V&Pynnl,s,Ioto!uvixwC-IJXElrgm C-.bcoCPJ
IMphsevhO AL!-K:AIkpre,
rPHEJUzV;P?uN3b?ohoRiBUENoV3B&jumNL;Aik,
xf -IEKROn JSyYWW?n 'ay;:weO'AqVzPyoiBL? seAX3Dot,iy.xyIcf r!!ul-Koi:x pZrAQly'v'a;vEzN
BwowKo'MBqF$PPFb
CjYX3beT,lZ qdda!wfgmJP
DUfNXmnQU mvcv?nlnQF$JUAAywNocd  bGSPyAlprNeQnq-GRSVUP.Ja!IBoDqfI&xJM AXEHV&DKvRS


## The mathematical trick in self-attention

In [18]:
# toy example illustrating how matrix multiplication can be used for a "weighted aggregation"
torch.manual_seed(42)
# Causal attention does not take into account future information
a = torch.tril(torch.ones(3, 3))
a = a / torch.sum(a, 1, keepdim=True)
b = torch.randint(0, 10, (3, 2)).float()
c = a @ b
print('a=')
print(a)
print('--')
print('b=')
print(b)
print('--')
print('c=')
print(c)

a=
tensor([[1.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000],
        [0.3333, 0.3333, 0.3333]])
--
b=
tensor([[2., 7.],
        [6., 4.],
        [6., 5.]])
--
c=
tensor([[2.0000, 7.0000],
        [4.0000, 5.5000],
        [4.6667, 5.3333]])


In [19]:
# consider the following toy example:

torch.manual_seed(1337)
B, T, C = 4, 8, 2  # batch, time, channels
x = torch.randn(B, T, C)
x.shape

torch.Size([4, 8, 2])

In [20]:
# We want $x[b,t] = mean_{i<=t} x[b,i]$
xbow = torch.zeros((B, T, C))
for b in range(B):
    for t in range(T):
        xprev = x[b, :t + 1]  # (t,C)
        xbow[b, t] = torch.mean(xprev, 0)

In [21]:
# version 2: using matrix multiply for a weighted aggregation
wei = torch.tril(torch.ones(T, T))
wei = wei / wei.sum(1, keepdim=True)
xbow2 = wei @ x  # (B, T, T) @ (B, T, C) ----> (B, T, C)
torch.allclose(xbow, xbow2)

False

In [22]:
# version 3: use Softmax
tril = torch.tril(torch.ones(T, T))
wei = torch.zeros((T, T))
# This seems strange at first glance but exp(-inf) = 0 and it is well defined, otherwise training would get broken constantly
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=-1)
xbow3 = wei @ x
torch.allclose(xbow, xbow3)

False

In [23]:
# version 4: self-attention!
torch.manual_seed(1337)
B, T, C = 4, 8, 32  # batch, time, channels
x = torch.randn(B, T, C)

# let's see a single Head perform self-attention
head_size = 16
key = nn.Linear(C, head_size, bias=False)
query = nn.Linear(C, head_size, bias=False)
value = nn.Linear(C, head_size, bias=False)
k = key(x)  # (B, T, 16)
q = query(x)  # (B, T, 16)
wei = q @ k.transpose(-2, -1)  # (B, T, 16) @ (B, 16, T) ---> (B, T, T)

tril = torch.tril(torch.ones(T, T))
# wei = torch.zeros((T,T))
# Full matrix multiplication is faster, just block what we do not need
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=-1)

v = value(x)
out = wei @ v
# out = wei @ x

out.shape

torch.Size([4, 8, 16])

In [24]:
wei[0]

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.1574, 0.8426, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2088, 0.1646, 0.6266, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5792, 0.1187, 0.1889, 0.1131, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0294, 0.1052, 0.0469, 0.0276, 0.7909, 0.0000, 0.0000, 0.0000],
        [0.0176, 0.2689, 0.0215, 0.0089, 0.6812, 0.0019, 0.0000, 0.0000],
        [0.1691, 0.4066, 0.0438, 0.0416, 0.1048, 0.2012, 0.0329, 0.0000],
        [0.0210, 0.0843, 0.0555, 0.2297, 0.0573, 0.0709, 0.2423, 0.2391]],
       grad_fn=<SelectBackward0>)

Notes:
- Attention is a **communication mechanism**. Can be seen as nodes in a directed graph looking at each other and aggregating information with a weighted sum from all nodes that point to them, with data-dependent weights.
- There is no notion of space. Attention simply acts over a set of vectors. This is why we need to positionally encode tokens.
- Each example across batch dimension is of course processed completely independently and never "talk" to each other
- In an "encoder" attention block just delete the single line that does masking with `tril`, allowing all tokens to communicate. This block here is called a "decoder" attention block because it has triangular masking, and is usually used in autoregressive settings, like language modeling.
- "self-attention" just means that the keys and values are produced from the same source as queries. In "cross-attention", the queries still get produced from x, but the keys and values come from some other, external source (e.g. an encoder module)
- "Scaled" attention additionally divides `wei` by 1/sqrt(head_size). This makes it so when input Q,K are unit variance, wei will be unit variance too and Softmax will stay diffuse and not saturate too much. Illustration below

In [25]:
# head_size is the dimension of "latent space"
k = torch.randn(B, T, head_size)
q = torch.randn(B, T, head_size)
wei = q @ k.transpose(-2, -1) * head_size ** -0.5

In [26]:
k.var()

tensor(1.0449)

In [27]:
q.var()

tensor(1.0700)

In [28]:
wei.var()

tensor(1.0918)

In [29]:
torch.softmax(torch.tensor([0.1, -0.2, 0.3, -0.2, 0.5]), dim=-1)

tensor([0.1925, 0.1426, 0.2351, 0.1426, 0.2872])

In [30]:
torch.softmax(torch.tensor([0.1, -0.2, 0.3, -0.2, 0.5]) * 8, dim=-1)  # gets too peaky, converges to one-hot

tensor([0.0326, 0.0030, 0.1615, 0.0030, 0.8000])

In [31]:
class LayerNorm1d:  # (used to be BatchNorm1d)

    def __init__(self, dim, eps=1e-5, momentum=0.1):
        self.eps = eps
        self.gamma = torch.ones(dim)
        self.beta = torch.zeros(dim)

    def __call__(self, x):
        # calculate the forward pass
        xmean = x.mean(1, keepdim=True)  # batch mean
        xvar = x.var(1, keepdim=True)  # batch variance
        xhat = (x - xmean) / torch.sqrt(xvar + self.eps)  # normalize to unit variance
        self.out = self.gamma * xhat + self.beta
        return self.out

    def parameters(self):
        return [self.gamma, self.beta]


torch.manual_seed(1337)
module = LayerNorm1d(100)
# batch_norm_1d = nn.BatchNorm1d(100)
# x_normalized_torch = batch_norm_1d(x)

x = torch.randn(32, 100)  # batch size 32 of 100-dimensional vectors
x_normalized = module(x)
x_normalized.shape

torch.Size([32, 100])

In [32]:
x[:, 0].mean(), x[:, 0].std()  # mean,std of one feature across all batch inputs

(tensor(0.1392), tensor(0.8899))

In [33]:
x[0, :].mean(), x[0, :].std()  # mean,std of a single input from the batch, of its features

(tensor(0.0409), tensor(1.0476))

In [34]:
# French to English translation example:

# <--------- ENCODE ------------------><--------------- DECODE ----------------->
# les réseaux de neurones sont géniaux! <START> neural networks are awesome!<END>

### Full finished code, for reference

You may want to refer directly to the git repo instead though.

In [35]:
import torch
import torch.nn as nn
from torch.nn import functional as F

# hyperparameters
batch_size = 1024 # 16  # how many independent sequences will we process in parallel?
block_size = 128 # 32  # what is the maximum context length for predictions?
max_iters = 10000
eval_interval = 100
learning_rate = 1e-3
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
n_embd = 64
n_head = 4
n_layer = 4
dropout = 0.0
# ------------

torch.manual_seed(1337)

# wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

# here are all the unique characters that occur in this text
chars = sorted(list(set(text)))
vocab_size = len(chars)
# create a mapping from characters to integers
stoi = {ch: i for i, ch in enumerate(chars)}
itos = {i: ch for i, ch in enumerate(chars)}
encode = lambda s: [stoi[c] for c in s]  # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l])  # decoder: take a list of integers, output a string

# Train and test splits
data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9 * len(data))  # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]


# data loading
def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i + block_size] for i in ix])
    y = torch.stack([data[i + 1:i + block_size + 1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y


@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out


class Head(nn.Module):
    """ one head of self-attention
        https://magazine.sebastianraschka.com/p/understanding-and-coding-self-attention
    """

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        # This limits us to the maximal context block_size
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B, T, C = x.shape
        k = self.key(x)  # (B,T,C=n_embd) -> (B,T,C=head_size)
        q = self.query(x)  # (B,T,C=n_embd) -> (B,T,C=head_size)
        # compute attention scores ("affinities")
        wei = q @ k.transpose(-2, -1) * C ** -0.5  # (B, T, C=head_size) @ (B, C=head_size), T) -> (B, T, T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf'))  # (B, T, T)
        wei = F.softmax(wei, dim=-1)  # (B, T, T)
        # The drop out is over full matrix, alternatively it would be better to just drop on mask, this is biased
        # Also it seems maybe conceptually we should just do symmetric dropout
        wei = self.dropout(wei)
        # perform the weighted aggregation of the values
        v = self.value(x)  # (B,T,C=head_size)
        # The matrix multiplication is batched and applied on last two dimensions!
        out = wei @ v  # (B, T, T) @ (B, T, C=head_size) -> (B, T, C=head_size)
        return out


class MultiHeadAttention(nn.Module):
    """ multiple heads of self-attention in parallel """

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        # Projection back to residual pathway, align the basis
        self.proj = nn.Linear(n_embd, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out


class FeedFoward(nn.Module):
    """ a simple linear layer followed by a non-linearity """

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd), # Projection back to residual pathway
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)


class Block(nn.Module):
    """ Transformer block: communication followed by computation """

    def __init__(self, n_embd: int, n_head: int):
        # n_embd: embedding dimension, n_head: the number of heads we'd like
        super().__init__()
        # To guarantee that final concatenated embedding is of size n_embd 
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedFoward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        # Skip connections added to improve the flow of gradient
        # Need to project back to residual pathway in sa and ffwd to "align bases" 
        
        # Modern way is do to layer norm before and not after, in original paper it was done after, we do before!
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x


# Not so super simple and not bigram model anymore
class BigramLanguageModel(nn.Module):

    def __init__(self):
        super().__init__()
        # each token is mapped to latent space of size n_embd
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        # learnable position embedding for positions 0, ..., block_size - 1
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        # Modern way of handling this is:
        # https://machinelearningmastery.com/a-gentle-introduction-to-positional-encoding-in-transformer-models-part-1/
        # https://afterhoursresearch.hashnode.dev/rope-rotary-positional-embedding
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd)  # final layer norm
        self.lm_head = nn.Linear(n_embd, vocab_size, bias=False)

    def forward(self, idx, targets=None):
        B, T = idx.shape

        # idx and targets are both (B,T) tensor of integers
        tok_emb = self.token_embedding_table(idx)  # (B,T,C=n_embd)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device))  # (T,C=n_embd)
        x = tok_emb + pos_emb  # (B,T,C=n_embd)
        x = self.blocks(x)  # (B,T,C=n_embd)
        # Modern way is do to layer norm before and not after, in original paper it was done after, we do before!
        x = self.ln_f(x)  # (B,T,C=n_embd)
        logits = self.lm_head(x)  # (B,T,vocab_size)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B * T, C)
            targets = targets.view(B * T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens, otherwise things will explode
            idx_cond = idx[:, -block_size:]
            # get the predictions
            logits, loss = self(idx_cond)
            # focus only on the last time step
            logits = logits[:, -1, :]  # (B, T, C=vocab_size) becomes (B, C=vocab_size)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1)  # (B, C=vocab_size)
            # sample from the multinomial distribution
            idx_next = torch.multinomial(probs, num_samples=1)  # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1)  # (B, T+1)
        return idx


model = BigramLanguageModel()
m = model.to(device)
# print the number of parameters in the model
print(sum(p.numel() for p in m.parameters()) / 1e6, 'M parameters')

# create a PyTorch optimizer
# https://www.ruder.io/optimizing-gradient-descent/#adam
# https://iclr-blogposts.github.io/2023/blog/2023/adamw/
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):

    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

# generate from the model
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decode(m.generate(context, max_new_tokens=2000)[0].tolist()))
# Temperature motivation https://en.wikipedia.org/wiki/LogSumExp
# What happens when you divide logits by posite Temp and then do softmax
# Temp close to 0 is max in limit, one hot, Temp >> 1 in limit random

0.215873 M parameters
step 0: train loss 4.3254, val loss 4.3198
step 100: train loss 2.5600, val loss 2.5620
step 200: train loss 2.4232, val loss 2.4276
step 300: train loss 2.2731, val loss 2.2882
step 400: train loss 2.1599, val loss 2.1881
step 500: train loss 2.0462, val loss 2.0977
step 600: train loss 1.9539, val loss 2.0254
step 700: train loss 1.8708, val loss 1.9669
step 800: train loss 1.8075, val loss 1.9265
step 900: train loss 1.7639, val loss 1.8996
step 1000: train loss 1.7138, val loss 1.8639
step 1100: train loss 1.6799, val loss 1.8366
step 1200: train loss 1.6495, val loss 1.8171
step 1300: train loss 1.6253, val loss 1.7901
step 1400: train loss 1.6056, val loss 1.7831
step 1500: train loss 1.5937, val loss 1.7649
step 1600: train loss 1.5675, val loss 1.7471
step 1700: train loss 1.5596, val loss 1.7430
step 1800: train loss 1.5432, val loss 1.7324
step 1900: train loss 1.5295, val loss 1.7198
step 2000: train loss 1.5209, val loss 1.7141
step 2100: train loss 1.

In [39]:
# https://github.com/szagoruyko/pytorchviz
# https://github.com/mert-kurttutan/torchview nicer graph
# https://github.com/mert-kurttutan/torchview
from torchview import draw_graph
import torchlens as tl

# device='meta' -> no memory is consumed for visualization
print(xb.shape)
print(yb.shape)
model_graph = draw_graph(model, input_data=(xb, yb), device='cuda')
model_graph.visual_graph

# model_history = tl.log_forward_pass(model, (xb, yb)
#                                      , layers_to_save='all', vis_opt='unrolled')
# print(model_history)

NameError: name 'model_history' is not defined

In [37]:
# generate from the model
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decode(model.generate(context, max_new_tokens=2000)[0].tolist()))


City and by his warms, went's scomptude, that
till ragge untong-make's.' But, what a determy, thou
faintors, you attend underpt my mother.

Second Men, an well.
Come I mean as thing once with I recond this;
And I may of looks to a plenity,
And fears 'free and joy myself,
When gazes liege titles wise and my restraint.

KING EDWARD IV:
Whoel, she was this head.

DUKE OF YORK:
Then frame thyself? for he forth and in this.
Were commend in the fiends, o' the voice
My mother counsished, to two Ely; the counter
That I do. So an ever maid in beast plots.

DUKE O FRIZWIIC John:
Upon thine is kind; too, Tower; thou dust baldgiant
And dost happille to the poore's doin work her,
Marriabe.

BENVOLIO:
If ends ye.

SICINIUS:
Good love I had unto conveys,
To my sovereign i' the dove me.

HENRY BOLINGHAM:
Were glast you shall be as the cannot both:
Thou art me in my consent until thee,
Or thee to direct against partiest;
But! seeing she parks of cross on me age good
So queen will answering doth chase,